In [1]:
import datetime
import os
import pandas as pd
from typing import Sequence

In [2]:
def read_partition(path: str) -> pd.DataFrame:
    def _read_partition(path: str, partition_cols: Sequence[Sequence[str]]) -> pd.DataFrame:
        if path.lower().endswith(".xlsx"):
            partitions = dict(partition_cols)
            if f"device{partitions["Device"]}" not in path:
                raise Exception(f"File {path} in wrong Device directory")
            df = pd.read_excel(path)
            return [ df.assign(**partitions) ]
        elif os.path.isdir(path):
            return [
                df
                for subpath in os.listdir(path)
                for df in _read_partition(f"{path}/{subpath}", partition_cols + ([subpath.split("=")] if "=" in subpath else []))
            ]
        else:
            return []
    
    dfs = _read_partition(path, [])
    return pd.concat(dfs) if dfs else pd.DataFrame()


def parse_tamtop_datetime(input: str) -> datetime.datetime:
    return datetime.datetime.fromisoformat(
        input.replace(" ", "T").replace("(", "").replace(")", "")
    )

In [3]:
df = read_partition("data")
df["Time"] = df["Time"].apply(parse_tamtop_datetime)
df = df.set_index(["Device", "Time"]).drop(["NO."], axis=1)
display(
    df.reset_index(level=1).groupby(level=0).agg(
        {
            "Time": ["min", "max", "count"],
            "Floor": ["nunique"],
            "Location": ["nunique"]
        }
    )
)

Time                                   Floor  \
                             min                       max count nunique   
Device                                                                     
01     2024-08-18 17:00:00+08:00 2024-08-21 15:00:00+08:00    64       1   
02     2024-08-18 17:00:00+08:00 2024-08-21 15:00:00+08:00    73       1   
03     2024-08-17 20:00:00+08:00 2024-08-21 15:00:00+08:00    82       1   

       Location  
        nunique  
Device           
01            2  
02            3  
03            3